In [1]:
import os
os.chdir('C:/Users/p_adi/OneDrive/Desktop')

In [5]:
import json
import pandas as pd
import glob
import numpy as np
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# Load the BIOasq data from the JSON file
with open('training9b.json', 'r') as f:
    data = json.load(f)

input_texts = []
output_texts = []

# Extract the input and output text from the data
for example in data['examples']:
    input_texts.append(example['input_text'])
    output_texts.append(example['output_text'])

# Tokenize the input and output text
tokenizer_inputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)

tokenizer_outputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_outputs.fit_on_texts(output_texts + ['<sos>'] + ['<eos>'])
output_sequences = tokenizer_outputs.texts_to_sequences(output_texts)

# Get the maximum sequence length for padding
max_input_len = max(len(s) for s in input_sequences)
max_output_len = max(len(s) for s in output_sequences)

# Pad the input and output sequences
input_sequences = pad_sequences(input_sequences, maxlen=max_input_len, padding='post')
output_sequences = pad_sequences(output_sequences, maxlen=max_output_len, padding='post')

# Define the input and output dimensions
input_dim = len(tokenizer_inputs.word_index) + 1
output_dim = len(tokenizer_outputs.word_index) + 1
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None, input_dim))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, output_dim))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the input and output sequences
model.fit([input_sequences, output_sequences[:, :-1]], output_sequences[:, 1:],
          batch_size=64, epochs=100, validation_split=0.2)

# Save the trained model
model.save('bioasq_seq2seq.h5')

# Define the encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Define the decoder states
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Get the decoder output
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# Define the decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_word_index = dict((i, word) for word, i in tokenizer_inputs.word_index.items())
reverse_output_word_index = dict((i, word) for word, i in tokenizer_outputs.word_index.items())



KeyError: 'examples'

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, output_dim))

    # Populate the first character of target sequence with the start character
    target_seq[0, 0, tokenizer_outputs.word_index['<sos>']] = 1

    # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_output_word_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop character
        if (sampled_word == '<eos>' or len(decoded_sentence) > max_output_len):
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np


In [ ]:


def generate_answers(dataframe):
    # Initialize an empty list to store the answers
    answers = []
    
    # Loop through each row of the dataframe
    for index, row in dataframe.iterrows():
        # Get the input question from the dataframe
        input_question = row['question']
        
        # Use the trained model to generate answers to new questions
        encoded_input_question = encode_input_sequence(input_question)
        decoded_answer = decode_sequence(encoded_input_question)
        
        # Append the answer to the list
        answers.append(decoded_answer)
        
    # Return the list of answers
    return answers

# Load the input data into a dataframe
df = pd.read_csv('input.csv')

# Generate answers for the questions in the dataframe
answers = generate_answers(df)

# Print the answers
print(answers)


In [6]:
import json

def extract_qa_pairs(json_data):
    qa_pairs = []
    for item in json_data:
        question = item.get("question")
        answer = item.get("answer")
        if question and answer:
            qa_pairs.append((question, answer))
    return qa_pairs

with open("training9b.json", "r") as f:
    json_data = json.load(f)
    qa_pairs = extract_qa_pairs(json_data)
    for q, a in qa_pairs:
        print("Question:", q)
        print("Answer:", a_dict[q])


AttributeError: 'str' object has no attribute 'get'

In [23]:
import pandas as pd
from simpletransformers.question_answering import QuestionAnsweringModel

def generate_questions(data, column_name):
    model = QuestionAnsweringModel('bert', 'bert-base-cased-distilled-squad', args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'use_cuda': False})
    questions = []
    for i in range(data.shape[0]):
        text = data[column_name].iloc[i]
        outputs = model.predict(text)
        questions.append(outputs['question'])
    return questions


In [24]:
import pandas as pd

data = pd.read_excel('qa_sample_data.xlsx')

In [25]:
generate_questions(data, 'context')

OSError: bert-base-uncased-distilled-squad is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [6]:
data.columns

Index(['disease', 'answer', 'context'], dtype='object')